# Competition Subject


## Detail contents



# Load Data

In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
feature = set(data.columns)
feat2 = []
feature.discard("TARGET")
feature.discard("ID")


# Feature Engineering

In [2]:
preprocess_list = []

## remove unique value Feature and if the values of test's feature is not unique, and don't discard


In [3]:
def removeOneValueFeature(data):
    for i in data.columns:
        data[i]
        if (len(data[i].unique()) == 1 and len(test[i].unique()) == 1):
            feature.discard(i)
            
            
removeOneValueFeature(data)
preprocess_list.append(removeOneValueFeature)

### 필요하면 여러 feature 들을 인코드한다.

In [40]:
from sklearn.svm import LinearSVC


from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel

feat = pd.Series(list(feature))

fin_data = data

def feature_selection(data):
    lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(data[feat], data["TARGET"])
    model = SelectFromModel(lsvc, prefit=True)
    new_feature = model.transform(data[feat])
    print len(lsvc.coef_[0])
    feat2 = []
    #print (lsvc.coef_[0])
    print (np.flatnonzero(lsvc.coef_))
    for i in np.flatnonzero(lsvc.coef_):
        feat2.append(feat[i])
            
    print len(feat2)
    
    return feat2
    
print len(feature)
feat2 = feature_selection(data)

335
335
[  0   2   7   9  11  14  15  16  17  22  24  25  31  32  33  36  40  43
  50  52  54  62  65  66  69  70  71  74  77  78  79  80  81  82  83  84
  86  87  89  91  96 101 106 109 111 112 115 118 123 124 125 128 129 130
 131 132 137 140 143 148 151 153 154 155 156 162 164 165 166 170 171 172
 173 174 175 178 180 181 184 185 189 190 192 193 195 197 198 205 208 209
 210 211 212 214 217 225 230 231 234 237 238 240 241 252 254 257 258 261
 263 265 270 271 272 273 283 284 285 286 290 291 295 298 303 307 309 310
 311 317 318 319 326 327 328 329 330 333]
136


# Cross validatae by ROCAUC



In [33]:


from sklearn.cross_validation import KFold
from sklearn import datasets
from sklearn.metrics import roc_auc_score



def rocauc_score(model, features, labels, num_folds = 5):
    kfolds = KFold(len(features), num_folds)
    
    total_score = 0.0
    
    for train_index, test_index in kfolds:
        train_features = features.iloc[train_index]
        test_features = features.iloc[test_index]
        train_labels = labels[train_index]
        test_labels = labels[test_index]
        
        model.fit(train_features, train_labels)
        prediction = model.predict_proba(test_features)
        
        
       
        nd = test_labels.get_values()
        pos_prediction = [x[1] for x in prediction]
        score = roc_auc_score(nd, pos_prediction)
        total_score = total_score + score
        
    total_score = total_score / num_folds
    
    return total_score
        
        
        

# Prediction

## Predict 

In [34]:
#from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import time


feature_names = pd.Series(list(feature))
feature_names2 = data.columns[1:370]
#배열로 넘김
label_name = "TARGET"
#컬럼 넘김
befo_time = time.time()
before = rocauc_score(RandomForestClassifier(n_estimators=200, n_jobs= -1), data[feature_names2], data[label_name])
befo_time = time.time() - befo_time

after_time = time.time()
after = rocauc_score(RandomForestClassifier(n_estimators=200, n_jobs= -1), data[feature_names], data[label_name])
after_time = time.time() - after_time


In [35]:
print("before = %.5f" % (before))
print befo_time
print ("after = %.5f" % (after) )
print after_time



before = 0.76073
54.8745250702
after = 0.76083
49.622177124


# Submit

In [36]:
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
feature = set(data.columns)

feature.discard("TARGET")
feature.discard("ID")
label_name = "TARGET"

for preprocess_function in preprocess_list:
    preprocess_function(train)
    





## Train

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import time


feature_names = pd.Series(feat2)
feature_names2 = train.columns[1:370]

print len(feature_names)
label_name = "TARGET"

#model = RandomForestClassifier(n_estimators = 200, n_jobs= -1)
start = time.time()
model = RandomForestClassifier(n_estimators=3500, n_jobs= -1)
#model = svm.SVC(probability = True)
model.fit(train[feature_names2], train[label_name])
prediction = model.predict_proba(test[feature_names2])
end = time.time() - start
print ("endt time ", end)


136
('endt time ', 245.7592179775238)


## Make submit

In [46]:
p = [x[1] for x in prediction]


submit = pd.DataFrame(data = {'TARGET' : p}, index = test["ID"] )
submit.index.names = ["ID"]
#submit.columns = label_columns

submit

,TARGET
ID,
2,0.148857
5,0.011714
6,0.000571
7,0.000000
9,0.000000
11,0.088000
12,0.066000
15,0.015143
16,0.016857


In [47]:
from time import strftime, localtime

current_time = strftime("%Y.%m.%d %H.%M.%S", localtime())

submit.to_csv("../submit/%s.csv" % current_time)